In [64]:
from src.experiment_based_function import SEEDS
import pandas as pd
%matplotlib inline

In [65]:
import os
import glob


In [66]:
eval_or_grid='eval'

In [67]:
def get_exp8_result(eval_or_grid='eval'):
    res = []    
    for fn in glob.glob('experiment_1001/exp8-spatial-thres-i/*/*/%s_res.csv' % eval_or_grid):
        _, i_thres, seed, _ = fn.split(os.path.sep)
        seed = seed.replace('seed_','')
        df = pd.read_csv(fn, index_col=0)
        df['seed'] = seed
        df['i_thres']=i_thres
        res.append(df)
    return pd.concat(res)

In [68]:
col_mapping = {' ': ' ',
 'RoadNet': 'RoadNet+Spatial',
 'Segment_NO_TOTAL_~2014+Spatial': 'Social type+Spatial', 
 'Segment_TOTAL_~2014+Spatial': 'Social total+Spatial',
 'RoadNet+Segment_NO_TOTAL_~2014+Spatial': 'RoadNet+Social type+Spatial', 
 'RoadNet+Segment_TOTAL_~2014+Spatial': 'RoadNet+Social total+Spatial'}


In [69]:
eval_res = get_exp8_result('eval')

eval_res.ftr_combo_name = eval_res.ftr_combo_name.apply(lambda x: col_mapping[x.split('-')[0]])

In [70]:
eval_res.columns, eval_res.shape

(Index(['#ftr_all', '#ftr_keep', '#test_sample', '#train_sample',
        'ftr_combo_name', 'model_name', 'test_acc', 'test_f1_macro',
        'test_f1_micro', 'test_f1_weighted', 'test_mse', 'test_n_classes',
        'test_precision_macro', 'test_precision_micro',
        'test_precision_weighted', 'test_recall_macro', 'test_recall_micro',
        'test_recall_weighted', 'train_acc', 'train_f1_macro', 'train_f1_micro',
        'train_f1_weighted', 'train_mse', 'train_n_classes',
        'train_precision_macro', 'train_precision_micro',
        'train_precision_weighted', 'train_recall_macro', 'train_recall_micro',
        'train_recall_weighted', 'y_dist', 'seed', 'i_thres'],
       dtype='object'), (600, 33))

In [71]:
print('train', eval_res.train_n_classes.value_counts().to_dict())
print('test', eval_res.test_n_classes.value_counts().to_dict())


print('train', 'test')
print(eval_res.apply(lambda x: (x.train_n_classes, x.test_n_classes), axis=1).value_counts())

train {5: 600}
test {5: 480, 4: 120}
train test
(5, 5)    480
(5, 4)    120
dtype: int64


In [72]:
f1_choices = ['test_f1_weighted', 'test_f1_macro', 'test_f1_micro']

In [73]:
keys = ['model_name', 'i_thres', 'ftr_combo_name']
means = eval_res.groupby(keys).mean()[f1_choices]
stds = eval_res.groupby(keys).std()[f1_choices]


In [74]:
means.test_f1_micro.max()

0.65505617977528074

In [75]:
def deliverable_table(table):
    table = table[['RoadNet+Spatial', 
                   'Social total+Spatial',
                   'Social type+Spatial',
                   'RoadNet+Social total+Spatial',
                   'RoadNet+Social type+Spatial']]

    table.index.name=None
    return table

In [76]:
from IPython.display import display

In [77]:
f1_type = f1_choices[2]
for i_thres in [0.1, 0.3, 0.5, 0.7, 0.8]:
    tmp = means.reset_index()
    table = tmp[tmp.i_thres==str(i_thres)].reset_index().pivot(index='model_name', columns='ftr_combo_name', values=f1_type)

    dtb = deliverable_table(table)
    dtb.columns.name=f1_type.replace('test_', '')+' i='+str(i_thres)
    display(dtb)


f1_micro i=0.1,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.619101,0.542697,0.597753,0.623596,0.623596
XGBcls,0.611236,0.538202,0.601124,0.621348,0.629213


f1_micro i=0.3,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.621348,0.542697,0.605618,0.631461,0.634831
XGBcls,0.606742,0.538202,0.607865,0.614607,0.642697


f1_micro i=0.5,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.606742,0.558427,0.619101,0.612360,0.641573
XGBcls,0.592135,0.559551,0.606742,0.624719,0.655056


f1_micro i=0.7,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.619101,0.564045,0.606742,0.62809,0.647191
XGBcls,0.574157,0.559551,0.606742,0.61236,0.647191


f1_micro i=0.8,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.606742,0.532584,0.616854,0.631461,0.638202
XGBcls,0.579775,0.547191,0.603371,0.634831,0.648315


In [78]:

tmp = means.reset_index()
table = tmp[tmp.i_thres=='Only'].reset_index().pivot(index='model_name', columns='ftr_combo_name', values=f1_type)

dtb = deliverable_table(table)
dtb.columns.name=f1_type.replace('test_', '')+' i='+str(i_thres)
display(dtb)

f1_micro i=0.8,RoadNet+Spatial,Social total+Spatial,Social type+Spatial,RoadNet+Social total+Spatial,RoadNet+Social type+Spatial
BAGcls,0.569663,0.540449,0.553933,0.557303,0.574157
XGBcls,0.571910,0.500000,0.548315,0.575281,0.565169
